In [32]:
# 4.3 LengthBasedExampleSelector
#  많은 prompt는 더 많은 비용을 지불해야한다. 지불할 돈이 있더라도 때로는 모델에 알맞는 양이 있다. 
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

from langchain.prompts.example_selector import LengthBasedExampleSelector
# LengthBasedExampleSelector는 기본적으로 예제들을 형식화 할 수 있고, 예제의 양이 얼마나 되는지 확인이 가능하다. 
#  내가 설정해둔 세팅값에 따라 prompt에 알맞는 예제를 골라준다. 

from langchain.prompts.example_selector.base import BaseExampleSelector
# 랜덤 예제 선택기를 만들기 위해서는 add_example 라는 method가 필요하다. (이미 존재하는 example에 example을 추가하는 method이다.)



chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

# 예제 선택기를 만드는 방법! 중 하나의 예! 
class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example): # example : : Dict[str, str]
        self.examples.append(examples)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]
# 선택한 예제 list를 반환해야하기 때문에. 무작위로 선택된 choice에 list로 넣어준다. 


example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}") 

# example_selector = LengthBasedExampleSelector(
#     examples=examples, 
#     example_prompt=example_prompt, #형식화와 양이 얼마나 되는지 알아보기 위해서 필요하다. 
#     max_length=180, #예제의 양을 얼마나 허용할 지를 정해줘야한다. 
# )
example_selector = RandomExampleSelector(
    examples=examples, 
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    # examples=examples, #예제의 양이 얼마나 되든 상관없이 !  example 호은 example_selector 둘 중 하나만 쓸 수 있다. 
    example_selector = example_selector, #예제의 양을 정해준다. 
    suffix="Human : What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="japan")

# chain = prompt| chat
# chain.invoke({
#     "country" : "Korea"
# })

'Human:What do you know about France?\nAI:\nHere is what I know:\nCapital: Paris\nLanguage: French\nFood: Wine and Cheese\nCurrency: Euro\n\n\nHuman : What do you know about japan?'